In [54]:
!pip install sqlparse --user --upgrade
!pip install sqlglot --user --upgrade

import sqlglot
import sqlglot.expressions as exp
from sqlglot import parse_one
from sqlglot.optimizer import optimize
from sqlglot import optimizer
from sqlglot.errors import OptimizeError

In [123]:
sql = '''
        
        insert into qma_datamart.otp_po_cut_level (
                                           COUNTRY_OF_ORIGIN,
                                           PO_CUT,
                                           STYLE,
                                           COLOR,
                                           STYLE_DESCRIPTION,
                                           GOODS_DESCRIPTION,
                                           PO_ISSUE_DATE,
                                           ORIGINAL_CRD_AT_ORIGIN,
                                           REVISED_CRD_AT_ORIGIN,
                                           ACTUAL_CRD_AT_ORIGIN,
                                           LOCAL_CURRENCY,
                                           --UNIT_PRICE,
                                           SEASON,
                                           SOURCING_OFFICE,
                                           SOURCE_SYSTEM,
                                           DC_CODE,
                                           PO_TYPE,
                                           PURCHASING_GROUP,
                                           SBU,
                                           SUB_SBU,
                                           PRODUCT_LINE,
                                           PURCHASING_COMPANY,
                                           VENDOR_FFC,
                                           VENDOR_GROUP_NAME,
                                           FACTORY_FFC,
                                           DELAY_REASON,
                                           SHIPMENT_TERMS,
                                           PO_LOCATION,
                                           VENDOR_NAME,
                                           FACTORY_NAME,
                                           REPORT_ORDER_QTY_LUM,
                                           ORDER_AMOUNT_LOCAL_CURRENCY,
                                           SHIPPED_QTY_LUM,
                                           SHIPPED_AMT_LOCAL_CURRENCY,
                                           BOOKED_QTY_LUM,
                                           BOOKED_AMT_LOCAL_CURRENCY,
                                           EFFECTIVE_QTY,
                                           isSample,
                                           isCustomOrder,
                                           MANAGING_OFFICE,
                                           "CURRENT_DATE",
                                           src_sys,
                                           MISC3,
                                           MISC26,
                                           MISC33,
                                           PURCHASING_COMPANY_CODE,
                                           exch_rate_date,
                                           od_misc_flag,
                                           tbr_defect_flag,
                                           po_creation_date,
                                           shipment_id,
                                           hts_code,
                                           factory_designation,
                                           ERP_Factory_Code,
                                           ERP_Vendor_Code,
                                           PO_Season,
                                           Revised_In_DC_Date,
                                           Ship_Mode,
                                           PO_Acknowledgement_Date,
                                           PO_Complete_Status,
                                           Greenlight_date,
                                           First_Shipment_ID,
                                           First_Actual_crd,
                                           master_po,
                                           freight_paid_by,
                                           exit_cnty_port,
                                           ship_to_name,
                                           Brand_Requested_in_DC_date,
                                           Original_Requested_Production_End_Date,
                                           Latest_confirmed_Production_End_Date,
                                           Buy_Month,
                                           requested_delivery_date,
                                           PO_closure_Eligibility,
                                           Discharge_at_Port_of_Destination,
										   season_cleaned --new add from 2766
                                           )
     select sp_country_master.name as COUNTRY_OF_ORIGIN,
            z.PO_CUT,
            z.STYLE,
            z.COLOR,
            z.STYLE_DESCRIPTION,
            z.GOODS_DESCRIPTION,
            z.PO_ISSUE_DATE,
            z.ORIGINAL_CRD_AT_ORIGIN,
            z.REVISED_CRD_AT_ORIGIN,
            max(z.ACTUAL_CRD_AT_ORIGIN),
            z.LOCAL_CURRENCY,
            z.SEASON,
            z.SOURCING_OFFICE,
            z.SOURCE_SYSTEM,
            z.DC_CODE,
            z.PO_TYPE,
            z.PURCHASING_GROUP,
            z.SBU,
            z.SUB_SBU,
            z.PRODUCT_LINE,
            dim_pch_grp.pg_name as PURCHASING_COMPANY,
            case when z.src_sys = 'WD' and nvl(vendor.vendor_ffc,'') = '' then factory_mapping.vendor_ffc else vendor.vendor_ffc end as VENDOR_FFC,
            upper(vendor.vendor_grp_name) as VENDOR_GROUP_NAME,
            case when z.src_sys = 'WD' and nvl(factory.fty_ffc,'') = '' then factory_mapping.factory_ffc else factory.fty_ffc end as FACTORY_FFC,
            z.DELAY_REASON,
            z.SHIPMENT_TERMS,
            z.PO_LOCATION,
            case when z.src_sys = 'WD' then vendor.vendor_grp_name else z.vendor_name end as VENDOR_NAME,
            case when z.src_Sys = 'WD' then factory.fty_name else z.fty_name end as FACTORY_NAME,
            sum(REPORT_ORDER_QTY_LUM) as REPORT_ORDER_QTY_LUM,
            sum(ORDER_AMOUNT_LC) as ORDER_AMOUNT_LC,
            sum(z.SHIPPED_QTY_LUM) as SHIPPED_QTY_LUM,
            sum(z.SHIPPED_AMT_LC) as SHIPPED_AMT_LC,
            sum(case when z.done_flag = true or z.SHIPPED_QTY_LUM > 0 then 0 else z.REPORT_ORDER_QTY_LUM end )  as BOOKED_QTY_LUM,
            sum(case when z.done_flag = true or z.SHIPPED_QTY_LUM > 0 then 0 else z.ORDER_AMOUNT_LC end) as BOOKED_AMT_LC,
            case when z.completed = true then sum(z.SHIPPED_QTY_LUM)
                 else sum(case when z.SHIPPED_QTY_LUM is null then z.REPORT_ORDER_QTY_LUM else z.SHIPPED_QTY_LUM end )
            end as EFFECTIVE_QTY,
            z.isSample,
            z.isCustomOrder,
            managing_office.managing_office as MANAGING_OFFICE,
            current_date as "CURRENT_DATE",
            z.src_sys as src_sys,
            case when z.src_sys = 'WD' then z.PRODUCT_LINE else z.MISC3 end as MISC3,
            z.MISC26,
            z.MISC33,
            z.PURCHASING_COMPANY,
            case when max(z.actual_crd_at_origin) is null or DATE_PART(year, max(z.actual_crd_at_origin)) = 1899 then max(z.revised_crd_at_origin) else max(z.actual_crd_at_origin) end as exch_rate_date,
            case when dim_pch_grp.pg_code in ('15101525','15051525','15501525') and dim_pch_grp.src_sys = 'OD' then 'True' 
                 else 'False' 
            end as od_misc_flag,
            case when (z.sub_sbu = 'TBL' and z.source_system='REVA' and substring(z.style,6,2)='FD') then 'True'
                 when (z.sbu like '%Timberland%' and z.source_system like '%JBA%' and substring(z.style,6,2)='FD') then 'True'
                 when (z.sbu not in ('TB PRO FOOTWEAR','TB TREE FOOTWEAR','TBL FOOTWEAR','CF TIMBERLAND FOOTWEAR','903','903 TBP_FOOTWEAR','906','906 TBT_FOOTWEAR') and z.dimension=2) then 'True'
                 else 'False'
            end as tbr_defect_flag,
            z.created_date as po_creation_date,
            max(z.shipment) as shipment_id,
            replace(z.hts_code,'.',''),
            ffc.designation as factory_designation,
            z.rfactory as ERP_Factory_Code,
            z.rda_code as ERP_Vendor_Code,
            z.PO_Season as PO_Season,
            z.revised_dc_date as Revised_In_DC_Date,
            z.shipment_mode as Ship_Mode,
            z.ack_date as PO_Acknowledgement_Date,
            z.done_flag as PO_Complete_Status,
            case when to_char(z.green_light_date,'yyyy')='1899' then null else z.green_light_date end as Greenlight_date,           
            min(z.First_Shipment_ID) as First_Shipment_ID,
            min(z.First_Actual_crd) as First_Actual_crd,
            z.master_po,
            z.freight_paid_by,
            z.exit_cnty_port,
            z.ship_to_name,
            z.Brand_Requested_in_DC_date,
            case when z.src_sys = 'OD' then z.misc22 else null end as Original_Requested_Production_End_Date,
            case when z.src_sys = 'OD' then z.misc30 else null end as Latest_confirmed_Production_End_Date,
            case when z.src_sys in ('OD','SW','WD') then z.misc24 else 'N/A' end as Buy_Month,
            case when z.src_sys in ('OD','SW','IW') then z.reqdelivery
                 when z.src_sys = 'WD' then null end as requested_delivery_date,
            case when z.src_sys in ('OD','SW','IW') then z.ship_route
                 when z.src_sys = 'WD' then 'Manual' end as PO_closure_Eligibility,
            null as Discharge_at_Port_of_Destination,
			z.season_cleaned
     from (select fact_po_hdr.lot as PO_CUT,
                  fact_po_hdr.style as STYLE,
                  fact_po_dtl.color as COLOR,
                  fact_po_hdr.desc as STYLE_DESCRIPTION,
                  fact_po_hdr.good_desc as GOODS_DESCRIPTION,
                  fact_po_hdr.issue_date as PO_ISSUE_DATE,
                  fact_po_hdr.orig_date as ORIGINAL_CRD_AT_ORIGIN,
                  fact_po_hdr.revised_date as REVISED_CRD_AT_ORIGIN,
                  case when max(nvl(fact_shp_dtl.actual_crd,'9999-12-31')) = '9999-12-31' then null else max(fact_shp_dtl.actual_crd) end as ACTUAL_CRD_AT_ORIGIN,
                  case when fact_po_hdr.currency is null or fact_po_hdr.currency = '' then 'USD' else fact_po_hdr.currency end as LOCAL_CURRENCY,
                  fact_po_hdr.season_cdf as SEASON,
                  regexp_replace(fact_po_hdr.source,'\\s+',' ') as SOURCING_OFFICE,
                  regexp_replace(fact_po_hdr.source_sys,'\\s+',' ') as SOURCE_SYSTEM,
                  regexp_replace(fact_po_hdr.store_no,'\\s+',' ') as DC_CODE,
                  fact_po_hdr.po_type as PO_TYPE,
                  regexp_replace(fact_po_hdr.purchasing_grp,'\\s+',' ') as PURCHASING_GROUP,
                  regexp_replace(fact_po_hdr.sbu,'\\s+',' ') as SBU,
                  regexp_replace(fact_po_hdr.sub_sbu,'\\s+',' ') as SUB_SBU,
                  --fact_po_hdr.cls_code as PRODUCT_LINE,
                  case when fact_po_hdr.src_sys = 'IW' then fact_po_hdr.cls_code
                       when fact_po_hdr.src_sys in ('SW','OD') then fact_po_hdr.pd_line
                   end as PRODUCT_LINE,
                  upper(fact_po_hdr.late_exit_rsn) as DELAY_REASON,
                  fact_po_hdr.shipment_term as SHIPMENT_TERMS,
                  fact_po_hdr.po_status as PO_LOCATION,
                  case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then (fact_po_dtl.qty * cast( nvl(dim_size_scale.scale_desc,'1') as int))
                      else fact_po_dtl.qty end as REPORT_ORDER_QTY_LUM,
                  fact_po_dtl.qty * fact_po_dtl.unit_cost as ORDER_AMOUNT_LC,
                  sum(case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then  cast( nvl(dim_size_scale.scale_desc,'1') as int) * fact_shp_dtl.ship_qty
                      else fact_shp_dtl.ship_qty end) as SHIPPED_QTY_LUM,
                  sum(fact_shp_dtl.ship_qty * fact_po_dtl.unit_cost) as SHIPPED_AMT_LC,
                  fact_po_dtl.unit_cost as unit_cost,
                  case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then  cast( nvl(dim_size_scale.scale_desc,'1') as int) else 1 end as Qty_per_Pack,
                  case when fact_po_hdr.src_sys != 'OD' and fact_po_hdr.po_type in ('CTL','SAM','SAN','AM') then TRUE
                  when fact_po_hdr.src_sys = 'OD' and fact_po_hdr.po_type in ('Z00S','ZSAM','ZTS','SAM','AM','ZSSD') then TRUE else FALSE end as isSample,
                  case when fact_po_hdr.src_sys = 'OD' and fact_po_hdr.po_type in ('ZCS','ZCUS','DYO') then TRUE else FALSE end as isCustomOrder,
                  fact_po_hdr.src_sys as src_sys,
                  regexp_replace(fact_po_hdr.MISC3,'\\s+',' ') as MISC3,
                  regexp_replace(fact_po_hdr.MISC26,'\\s+',' ') as MISC26,
                  regexp_replace(fact_po_hdr.MISC33,'\\s+',' ') as MISC33,
                  regexp_replace(fact_po_hdr.PURCHASING_COMPANY,'\\s+',' ') as PURCHASING_COMPANY,
                  fact_po_hdr.rfactory as rfactory,
                  fact_po_hdr.rda_code,
                  fact_po_hdr.season as PO_Season,
                  fact_po_hdr.vendor_name ,
                  fact_po_hdr.fty_name,
                  fact_po_dtl.dimension,
                  fact_po_hdr.created_date,
                  max(fact_shp_hdr.shipment) as shipment,
                  fact_po_hdr.hts_code as hts_code,
                  cast(fact_po_hdr.revised_dc_date as date) as revised_dc_date,
                  fact_po_hdr.shipment_mode,
                  cast(fact_po_hdr.ack_date as date) as ack_date,
                  fact_po_hdr.done_flag,
                  cast(fact_po_hdr.green_light_date as date) as green_light_date,
                  min(fact_shp_hdr.shipment)as First_Shipment_ID,
                  min(cast(fact_shp_dtl.actual_crd as date)) as First_Actual_crd,
                  fact_po_hdr.completed,
                  fact_po_hdr.ship_no as master_po,
                  fact_po_hdr.freight_paid_by, -- for shipment forecast
                  case when nvl(fact_po_hdr.exit_cnty_port, fact_po_hdr.exit_cnty_port33) ='' then  fact_po_hdr.exit_cnty_port33 else fact_po_hdr.exit_cnty_port end as exit_cnty_port, -- for shipment forecast
                  case when fact_po_hdr.store_no in ('1009','1014','R009') and CHARINDEX('P2:',fact_po_hdr.notes ) > 0 and CHARINDEX('P3:',fact_po_hdr.notes ) >0 then SUBSTRING(fact_po_hdr.notes ,CHARINDEX('P2:',fact_po_hdr.notes )+3,CHARINDEX('P3:',fact_po_hdr.notes )-CHARINDEX('P2:', fact_po_hdr.notes )-3) end  as ship_to_name, -- for shipment forecast
                  fact_po_hdr.orig_dc as Brand_Requested_in_DC_date,
                  regexp_replace(fact_po_hdr.req_production_date,'\\s+',' ') as MISC22,
                  regexp_replace(fact_po_hdr.confirmed_pd_date,'\\s+',' ') as MISC30,
                  regexp_replace(fact_po_hdr.buy_month,'\\s+',' ') as MISC24,
                  cast(fact_po_hdr.reqdelivery as date) as reqdelivery,
                  case when fact_po_hdr.ship_route in ('3PL','TLO') then 'Auto' 
                       when fact_po_hdr.ship_route in ('NGC','TLP') then 'Manual' else 'Manual' end as ship_route ,
				  fact_po_hdr.season_cleaned
           FROM qma.fact_po_hdr
           JOIN qma.fact_po_dtl
             ON fact_po_hdr.id_cut = fact_po_dtl.id_cut and fact_po_hdr.src_sys = fact_po_dtl.src_sys
           LEFT JOIN qma.fact_shp_dtl
             ON fact_po_hdr.season = fact_shp_dtl.season
             AND fact_po_hdr.style = fact_shp_dtl.style
             AND fact_po_hdr.lot = fact_shp_dtl.lot
             AND fact_po_dtl.color = fact_shp_dtl.color
             AND fact_po_dtl.size = fact_shp_dtl.size
             AND fact_po_dtl.dimension = fact_shp_dtl.dimension
             AND fact_po_dtl.src_sys = fact_shp_dtl.src_sys
             AND fact_shp_dtl.changeoperation <> 'D'
           LEFT JOIN qma.fact_shp_hdr
             ON fact_shp_dtl.shipment = fact_shp_hdr.shipment
             and fact_shp_dtl.src_sys = fact_shp_hdr.src_sys
             and fact_shp_hdr.changeoperation<>'D'
           LEFT JOIN qma.dim_size_scale
             ON fact_po_dtl.size = dim_size_scale.scale_code 
             and fact_po_dtl.src_sys = dim_size_scale.src_sys 
             and dim_size_scale.scale_desc != '^[0-9]+$'
           where fact_po_hdr.issue_date >='2019-4-1'  
             and fact_po_hdr.changeoperation <>'D' and fact_po_dtl.changeoperation <>'D' 
             and fact_po_hdr.src_sys <> 'WD'
             and upper(nvl(fact_po_hdr.po_status,'')) <> 'CANCELED'
             and not (fact_po_hdr.done_flag = true and fact_po_hdr.ship_qty = 0)
             and not (fact_po_dtl.qty = 0 and fact_po_dtl.ship_qty = 0)
             and not (fact_po_dtl.ship_qty > 0 and (fact_shp_dtl.actual_crd is null or DATE_PART_YEAR(cast (fact_shp_dtl.actual_crd as date)) < 1900))
           group by fact_po_hdr.lot,
                    fact_po_hdr.style ,
                    fact_po_dtl.color ,
                    fact_po_hdr.desc ,
                    fact_po_hdr.good_desc ,
                    fact_po_hdr.issue_date ,
                    fact_po_hdr.orig_date ,
                    fact_po_hdr.revised_date ,
                    case when fact_po_hdr.currency is null or fact_po_hdr.currency = '' then 'USD' else fact_po_hdr.currency end,
                    fact_po_hdr.season_cdf ,
                    regexp_replace(fact_po_hdr.source,'\\s+',' '),
                    regexp_replace(fact_po_hdr.source_sys,'\\s+',' '),
                    regexp_replace(fact_po_hdr.store_no,'\\s+',' '),
                    fact_po_hdr.po_type,
                    regexp_replace(fact_po_hdr.purchasing_grp,'\\s+',' '),
                    regexp_replace(fact_po_hdr.sbu,'\\s+',' '),
                    regexp_replace(fact_po_hdr.sub_sbu,'\\s+',' '),
                    --fact_po_hdr.cls_code,
                    case when fact_po_hdr.src_sys = 'IW' then fact_po_hdr.cls_code
                         when fact_po_hdr.src_sys in ('SW','OD') then fact_po_hdr.pd_line
                     end,
                    upper(fact_po_hdr.late_exit_rsn) ,
                    fact_po_hdr.shipment_term ,
                    fact_po_hdr.po_status ,
                    case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then (fact_po_dtl.qty * cast( nvl(dim_size_scale.scale_desc,'1') as int))
                         else fact_po_dtl.qty end,
                    fact_po_dtl.qty * fact_po_dtl.unit_cost,
                    fact_po_dtl.unit_cost,
                    case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then  cast( nvl(dim_size_scale.scale_desc,'1') as int) else 1 end ,
                    case when fact_po_hdr.src_sys != 'OD' and fact_po_hdr.po_type in ('CTL','SAM','SAN','AM') then TRUE 
                    when fact_po_hdr.src_sys = 'OD' and fact_po_hdr.po_type in ('Z00S','ZSAM','ZTS','SAM','AM','ZSSD') then TRUE else FALSE end,
                    case when fact_po_hdr.src_sys = 'OD' and fact_po_hdr.po_type in ('ZCS','ZCUS','DYO') then TRUE else FALSE end,
                    fact_po_hdr.src_sys,
                    regexp_replace(fact_po_hdr.MISC3,'\\s+',' '),
                    regexp_replace(fact_po_hdr.MISC26,'\\s+',' '),
                    regexp_replace(fact_po_hdr.MISC33,'\\s+',' '),
                    regexp_replace(fact_po_hdr.PURCHASING_COMPANY,'\\s+',' '),
                    fact_po_hdr.rfactory,
                    fact_po_hdr.rda_code,
                    fact_po_hdr.season,
                    fact_po_hdr.vendor_name,
                    fact_po_hdr.fty_name,
                    fact_po_dtl.po_dtl_seq,
                    fact_po_dtl.dimension,
                    fact_po_hdr.created_date,
                    fact_po_hdr.hts_code,
                    cast(fact_po_hdr.revised_dc_date as date),
                    fact_po_hdr.shipment_mode,
                    cast(fact_po_hdr.ack_date as date),
                    fact_po_hdr.done_flag,
                    cast(fact_po_hdr.green_light_date as date),
                    fact_po_hdr.completed,
                    fact_po_hdr.ship_no,
                    fact_po_hdr.freight_paid_by, 
                    case when nvl(fact_po_hdr.exit_cnty_port, fact_po_hdr.exit_cnty_port33) ='' then  fact_po_hdr.exit_cnty_port33 else fact_po_hdr.exit_cnty_port end, -- for shipment forecast
                    case when fact_po_hdr.store_no in ('1009','1014','R009') and CHARINDEX('P2:',fact_po_hdr.notes ) > 0 and CHARINDEX('P3:',fact_po_hdr.notes ) >0 then SUBSTRING(fact_po_hdr.notes ,CHARINDEX('P2:',fact_po_hdr.notes )+3,CHARINDEX('P3:',fact_po_hdr.notes )-CHARINDEX('P2:', fact_po_hdr.notes )-3) end, -- for shipment forecast
                    fact_po_hdr.orig_dc,
                    regexp_replace(fact_po_hdr.req_production_date,'\\s+',' '),
                    regexp_replace(fact_po_hdr.confirmed_pd_date,'\\s+',' '),
                    regexp_replace(fact_po_hdr.buy_month,'\\s+',' '),
                    cast(fact_po_hdr.reqdelivery as date),
                    case when fact_po_hdr.ship_route in ('3PL','TLO') then 'Auto' 
                         when fact_po_hdr.ship_route in ('NGC','TLP') then 'Manual' else 'Manual' end,
				    fact_po_hdr.season_cleaned
           UNION ALL
           select fact_po_hdr.lot as PO_CUT,
                  fact_po_hdr.style as STYLE,
                  fact_po_dtl.color as COLOR,
                  fact_po_hdr.desc as STYLE_DESCRIPTION,
                  fact_po_hdr.good_desc as GOODS_DESCRIPTION,
                  fact_po_hdr.issue_date as PO_ISSUE_DATE,
                  fact_po_hdr.orig_date as ORIGINAL_CRD_AT_ORIGIN,
                  fact_po_hdr.revised_date as REVISED_CRD_AT_ORIGIN,
                  case when max(nvl(fact_shp_dtl.actual_crd,'9999-12-31')) in ('9999-12-31','1900-01-01') then null else max(fact_shp_dtl.actual_crd) end as ACTUAL_CRD_AT_ORIGIN,
                  case when fact_po_hdr.currency is null or fact_po_hdr.currency = '' then 'USD' else fact_po_hdr.currency end as LOCAL_CURRENCY,
                  fact_po_hdr.season_cdf as SEASON,
                  regexp_replace(fact_po_hdr.source,'\\s+',' ') as SOURCING_OFFICE,
                  regexp_replace(fact_po_hdr.source_sys,'\\s+',' ') as SOURCE_SYSTEM,
                  regexp_replace(fact_po_hdr.store_no,'\\s+',' ') as DC_CODE,
                  fact_po_hdr.po_type as PO_TYPE,
                  regexp_replace(fact_po_hdr.purchasing_grp,'\\s+',' ') as PURCHASING_GROUP,
                  regexp_replace(fact_po_hdr.sbu,'\\s+',' ') as SBU,
                  regexp_replace(fact_po_hdr.sub_sbu,'\\s+',' ') as SUB_SBU,
                  fact_po_hdr.pd_line as PRODUCT_LINE,
                  upper(fact_po_hdr.late_exit_rsn) as DELAY_REASON,
                  fact_po_hdr.shipment_term as SHIPMENT_TERMS,
                  fact_po_hdr.po_status as PO_LOCATION,
                  case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then (fact_po_dtl.qty * cast( nvl(dim_size_scale.scale_desc,'1') as int))
                      else fact_po_dtl.qty end as REPORT_ORDER_QTY_LUM,
                  fact_po_dtl.qty * fact_po_dtl.unit_cost as ORDER_AMOUNT_LC,
                  sum(case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then  cast( nvl(dim_size_scale.scale_desc,'1') as int) * fact_shp_dtl.ship_qty
                      else fact_shp_dtl.ship_qty end) as SHIPPED_QTY_LUM,
                  sum(fact_shp_dtl.ship_qty * fact_po_dtl.unit_cost) as SHIPPED_AMT_LC,
                  fact_po_dtl.unit_cost as unit_cost,
                  case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then  cast( nvl(dim_size_scale.scale_desc,'1') as int) else 1 end as Qty_per_Pack,
                  case when fact_po_hdr.po_type in ('CTL','SAM','SAN','AM') then TRUE else FALSE end as isSample,
                  FALSE as isCustomOrder,
                  fact_po_hdr.src_sys as src_sys,
                  regexp_replace(fact_po_hdr.MISC3,'\\s+',' ') as MISC3,
                  regexp_replace(fact_po_hdr.MISC26,'\\s+',' ') as MISC26,
                  regexp_replace(fact_po_hdr.MISC33,'\\s+',' ') as MISC33,
                  regexp_replace(fact_po_hdr.PURCHASING_COMPANY,'\\s+',' ') as PURCHASING_COMPANY,
                  fact_po_hdr.rfactory,
                  fact_po_hdr.rda_code,
                  fact_po_hdr.season as PO_Season,
                  fact_po_hdr.vendor_name,
                  fact_po_hdr.fty_name,
                  fact_po_dtl.dimension,
                  fact_po_hdr.created_date,
                  max(fact_shp_hdr.shipment) as shipment,
                  fact_po_hdr.hts_code as hts_code,
                  cast(fact_po_hdr.revised_dc_date as date)as revised_dc_date,
                  fact_po_hdr.shipment_mode,
                  cast(fact_po_hdr.ack_date as date) as ack_date,
                  fact_po_hdr.done_flag,
                  cast(fact_po_hdr.green_light_date as date) as green_light_date,
                  min(fact_shp_hdr.shipment)as First_Shipment_ID,
                  min(cast(fact_shp_dtl.actual_crd as date)) as First_Actual_crd,
                  fact_po_hdr.completed,
                  fact_po_hdr.ship_no as master_po,
                  fact_po_hdr.freight_paid_by, -- for shipment forecast
                  case when nvl(fact_po_hdr.exit_cnty_port, fact_po_hdr.exit_cnty_port33) ='' then  fact_po_hdr.exit_cnty_port33 else fact_po_hdr.exit_cnty_port end as exit_cnty_port, -- for shipment forecast
                  case when fact_po_hdr.store_no in ('1009','1014','R009') and CHARINDEX('P2:',fact_po_hdr.notes ) > 0 and CHARINDEX('P3:',fact_po_hdr.notes ) >0 then SUBSTRING(fact_po_hdr.notes ,CHARINDEX('P2:',fact_po_hdr.notes )+3,CHARINDEX('P3:',fact_po_hdr.notes )-CHARINDEX('P2:', fact_po_hdr.notes )-3) end  as ship_to_name, -- for shipment forecast
                  fact_po_hdr.orig_dc as Brand_Requested_in_DC_date,
                  regexp_replace(fact_po_hdr.req_production_date,'\\s+',' ') as MISC22,
                  regexp_replace(fact_po_hdr.confirmed_pd_date,'\\s+',' ') as MISC30,
                  regexp_replace(fact_po_hdr.buy_month,'\\s+',' ') as MISC24,
                  null as reqdelivery,
                  case when fact_po_hdr.ship_route in ('3PL','TLO') then 'Auto' 
                       when fact_po_hdr.ship_route in ('NGC','TLP') then 'Manual' else 'Manual' end as ship_route,
			      fact_po_hdr.season_cleaned
           FROM qma.fact_po_hdr
           JOIN qma.fact_po_dtl
             ON fact_po_hdr.id_cut = fact_po_dtl.id_cut and fact_po_hdr.src_sys = fact_po_dtl.src_sys
           LEFT JOIN qma.fact_shp_dtl
             ON fact_po_hdr.season = fact_shp_dtl.season 
             AND fact_po_hdr.style = fact_shp_dtl.style 
             AND fact_po_hdr.id_cut = fact_shp_dtl.id_cut
             AND fact_po_dtl.color = fact_shp_dtl.color 
             AND fact_po_dtl.size = fact_shp_dtl.size 
             AND fact_po_dtl.dimension = fact_shp_dtl.dimension 
             AND fact_po_dtl.src_sys = fact_shp_dtl.src_sys 
             AND fact_shp_dtl.changeoperation <> 'D'
           LEFT JOIN qma.fact_shp_hdr
             ON fact_shp_dtl.shipment = fact_shp_hdr.shipment 
             and fact_shp_dtl.src_sys = fact_shp_hdr.src_sys 
             and fact_shp_hdr.changeoperation<>'D'
           LEFT JOIN qma.dim_size_scale
             ON fact_po_dtl.size = dim_size_scale.scale_code 
             and fact_po_dtl.src_sys = dim_size_scale.src_sys 
             and dim_size_scale.scale_desc != '^[0-9]+$'
           where fact_po_hdr.issue_date >='2019-4-1'  
             and fact_po_hdr.changeoperation <>'D' and fact_po_dtl.changeoperation <>'D' 
             and fact_po_hdr.src_sys = 'WD'
             and upper(nvl(fact_po_hdr.po_status,'')) <> 'CANCELED'
           group by fact_po_hdr.lot,
                    fact_po_hdr.style ,
                    fact_po_dtl.color,
                    fact_po_hdr.desc ,
                    fact_po_hdr.good_desc ,
                    fact_po_hdr.issue_date ,
                    fact_po_hdr.orig_date ,
                    fact_po_hdr.revised_date ,
                    case when fact_po_hdr.currency is null or fact_po_hdr.currency = '' then 'USD' else fact_po_hdr.currency end,
                    fact_po_hdr.season_cdf ,
                    regexp_replace(fact_po_hdr.source,'\\s+',' '),
                    regexp_replace(fact_po_hdr.source_sys,'\\s+',' '),
                    regexp_replace(fact_po_hdr.store_no,'\\s+',' '),
                    fact_po_hdr.po_type,
                    regexp_replace(fact_po_hdr.purchasing_grp,'\\s+',' '),
                    regexp_replace(fact_po_hdr.sbu,'\\s+',' '),
                    regexp_replace(fact_po_hdr.sub_sbu,'\\s+',' '),
                    fact_po_hdr.pd_line,
                    upper(fact_po_hdr.late_exit_rsn) ,
                    fact_po_hdr.shipment_term ,
                    fact_po_hdr.po_status ,
                    case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then (fact_po_dtl.qty * cast( nvl(dim_size_scale.scale_desc,'1') as int))
                         else fact_po_dtl.qty end,
                    fact_po_dtl.qty * fact_po_dtl.unit_cost,
                    fact_po_dtl.unit_cost,
                    case when length(fact_po_hdr.style)=12 and right(fact_po_hdr.style,1)='P' then  cast( nvl(dim_size_scale.scale_desc,'1') as int) else 1 end ,
                    case when fact_po_hdr.po_type in ('CTL','SAM','SAN','AM') then TRUE else FALSE end,
                    fact_po_hdr.src_sys,
                    regexp_replace(fact_po_hdr.MISC3,'\\s+',' '),
                    regexp_replace(fact_po_hdr.MISC26,'\\s+',' '),
                    regexp_replace(fact_po_hdr.MISC33,'\\s+',' '),
                    regexp_replace(fact_po_hdr.PURCHASING_COMPANY,'\\s+',' '),
                    fact_po_hdr.rfactory,
                    fact_po_hdr.rda_code,
                    fact_po_hdr.season ,
                    fact_po_hdr.vendor_name,
                    fact_po_hdr.fty_name,
                    fact_po_dtl.po_dtl_seq,
                    fact_po_dtl.dimension,
                    fact_po_hdr.created_date,
                    fact_po_hdr.hts_code,
                    cast(fact_po_hdr.revised_dc_date as date),
                    fact_po_hdr.shipment_mode,
                    cast(fact_po_hdr.ack_date as date),
                    fact_po_hdr.done_flag,
                    cast(fact_po_hdr.green_light_date as date),
                    fact_po_hdr.completed,
                    fact_po_hdr.ship_no,
                    fact_po_hdr.freight_paid_by, -- for shipment forecast
                    case when nvl(fact_po_hdr.exit_cnty_port, fact_po_hdr.exit_cnty_port33) ='' then  fact_po_hdr.exit_cnty_port33 else fact_po_hdr.exit_cnty_port end, -- for shipment forecast
                    case when fact_po_hdr.store_no in ('1009','1014','R009') and CHARINDEX('P2:',fact_po_hdr.notes ) > 0 and CHARINDEX('P3:',fact_po_hdr.notes ) >0 then SUBSTRING(fact_po_hdr.notes ,CHARINDEX('P2:',fact_po_hdr.notes )+3,CHARINDEX('P3:',fact_po_hdr.notes )-CHARINDEX('P2:', fact_po_hdr.notes )-3) end, -- for shipment forecast
                    fact_po_hdr.orig_dc,
                    regexp_replace(fact_po_hdr.req_production_date,'\\s+',' '),
                    regexp_replace(fact_po_hdr.confirmed_pd_date,'\\s+',' '),
                    regexp_replace(fact_po_hdr.buy_month,'\\s+',' '),
                    case when fact_po_hdr.ship_route in ('3PL','TLO') then 'Auto' 
                         when fact_po_hdr.ship_route in ('NGC','TLP') then 'Manual' else 'Manual' end,
				    fact_po_hdr.season_cleaned                    
     )z
     LEFT JOIN qma.dim_fty_supr vendor
       ON z.rda_code = vendor.fty_code 
       and z.src_sys = vendor.src_sys
       and vendor.changeoperation <> 'D'
     LEFT JOIN qma.dim_fty_supr factory
       ON z.rfactory = factory.fty_code 
       and z.src_sys = factory.src_sys
       and factory.changeoperation <> 'D'
     LEFT JOIN qma.dim_pch_grp
       ON z.purchasing_company = dim_pch_grp.pg_code 
       and z.src_sys = dim_pch_grp.src_sys 
     LEFT JOIN qma_datamart.mapping_crc_grouping crc
       ON z.DELAY_REASON = upper(crc.old_delay_reason_code)||' '||upper(crc.old_delay_category_1)
     LEFT JOIN qma_datamart.sp_managing_office_mapping managing_office
       ON upper(z.rfactory) =  upper(managing_office.erp_factory_code)
    
     left JOIN qma_datamart.sp_country_master
       on left(factory.un_loc_code,2) = sp_country_master.code                      
     left join qma_datamart.mapping_ngcv12_factory factory_mapping
       on z.rfactory = 'WD'||factory_mapping.factory
     left JOIN qma.ffc_factory_designation_status ffc
       on ffc.acc_id=(case when z.src_sys='WD' and nvl(factory.fty_ffc,'')='' then factory_mapping.factory_ffc else factory.fty_ffc end) 
       and cast(z.PO_ISSUE_DATE as date) between ffc.designation_eff_on and ffc.designation_exp_date 
     group by sp_country_master.name,
              z.PO_CUT,
              z.STYLE,
              z.COLOR,
              z.STYLE_DESCRIPTION,
              z.GOODS_DESCRIPTION,
              z.PO_ISSUE_DATE,
              z.ORIGINAL_CRD_AT_ORIGIN,
              z.REVISED_CRD_AT_ORIGIN,
              --z.ACTUAL_CRD_AT_ORIGIN,
              z.LOCAL_CURRENCY,
              z.SEASON,
              z.SOURCING_OFFICE,
              z.SOURCE_SYSTEM,
              z.DC_CODE,
              z.PO_TYPE,
              z.PURCHASING_GROUP,
              z.SBU,
              z.SUB_SBU,
              z.PRODUCT_LINE,
              dim_pch_grp.pg_name,
              case when z.src_sys = 'WD' and nvl(vendor.vendor_ffc,'') = '' then factory_mapping.vendor_ffc else vendor.vendor_ffc end,
              vendor.vendor_grp_name,
              case when z.src_sys = 'WD' and nvl(factory.fty_ffc,'') = '' then factory_mapping.factory_ffc else factory.fty_ffc end,
              z.DELAY_REASON,
              z.SHIPMENT_TERMS,
              z.PO_LOCATION,
              vendor.fty_comm_name,
              factory.fty_comm_name,
              z.isSample,
              z.isCustomOrder,
              managing_office.managing_office,
              
              z.src_sys,
              --for auap-2895
              case when z.src_sys = 'WD' then z.PRODUCT_LINE else z.MISC3 end,
              z.MISC26,
              z.MISC33,
              z.PURCHASING_COMPANY,
              case when z.src_sys = 'WD' then vendor.vendor_grp_name else z.vendor_name end,
              case when z.src_Sys = 'WD' then factory.fty_name else z.fty_name end,
              case when dim_pch_grp.pg_code in ('15101525','15051525','15501525') and dim_pch_grp.src_sys = 'OD' then 'True' 
              else 'False' end,
              case when (z.sub_sbu = 'TBL' and z.source_system='REVA' and substring(z.style,6,2)='FD') then 'True'
                   when (z.sbu like '%Timberland%' and z.source_system like '%JBA%' and substring(z.style,6,2)='FD') then 'True'
                   when (z.sbu not in ('TB PRO FOOTWEAR','TB TREE FOOTWEAR','TBL FOOTWEAR','CF TIMBERLAND FOOTWEAR','903','903 TBP_FOOTWEAR','906','906 TBT_FOOTWEAR') and z.dimension=2) then 'True'
                   else 'False'
              end,
              z.created_date,
              replace(z.hts_code,'.',''),
              ffc.designation,
              z.rda_code,
              z.PO_Season,
              z.rfactory,
              z.revised_dc_date,
              z.shipment_mode,
              z.ack_date,
              --z.completed_date,
              z.done_flag,
              z.green_light_date,
              z.completed,
              z.master_po,
              z.freight_paid_by,
              z.exit_cnty_port,
              z.ship_to_name,
              z.Brand_Requested_in_DC_date,
              case when z.src_sys = 'OD' then z.misc22 else null end,
              case when z.src_sys = 'OD' then z.misc30 else null end,
              case when z.src_sys in ('OD','SW','WD') then z.misc24 else 'N/A' end,
              case when z.src_sys in ('OD','SW','IW') then z.reqdelivery
                   when z.src_sys = 'WD' then null end,
              case when z.src_sys in ('OD','SW','IW') then z.ship_route
                   when z.src_sys = 'WD' then 'Manual' end,
			  z.season_cleaned
              ; '''

In [162]:
sql

'\n        \n        insert into qma_datamart.otp_po_cut_level (\n                                           COUNTRY_OF_ORIGIN,\n                                           PO_CUT,\n                                           STYLE,\n                                           COLOR,\n                                           STYLE_DESCRIPTION,\n                                           GOODS_DESCRIPTION,\n                                           PO_ISSUE_DATE,\n                                           ORIGINAL_CRD_AT_ORIGIN,\n                                           REVISED_CRD_AT_ORIGIN,\n                                           ACTUAL_CRD_AT_ORIGIN,\n                                           LOCAL_CURRENCY,\n                                           --UNIT_PRICE,\n                                           SEASON,\n                                           SOURCING_OFFICE,\n                                           SOURCE_SYSTEM,\n                                         

In [124]:
#json format
ast = parse_one(sql)
print(repr(ast))

Insert(
  this=Schema(
    this=Table(
      this=Identifier(this=otp_po_cut_level, quoted=False),
      db=Identifier(this=qma_datamart, quoted=False)),
    expressions=[
      Identifier(this=COUNTRY_OF_ORIGIN, quoted=False),
      Identifier(this=PO_CUT, quoted=False),
      Identifier(this=STYLE, quoted=False),
      Identifier(this=COLOR, quoted=False),
      Identifier(this=STYLE_DESCRIPTION, quoted=False),
      Identifier(this=GOODS_DESCRIPTION, quoted=False),
      Identifier(this=PO_ISSUE_DATE, quoted=False),
      Identifier(this=ORIGINAL_CRD_AT_ORIGIN, quoted=False),
      Identifier(this=REVISED_CRD_AT_ORIGIN, quoted=False),
      Identifier(this=ACTUAL_CRD_AT_ORIGIN, quoted=False),
      Identifier(this=LOCAL_CURRENCY, quoted=False),
      Identifier(this=SEASON, quoted=False, _comments=[
          UNIT_PRICE,]),
      Identifier(this=SOURCING_OFFICE, quoted=False),
      Identifier(this=SOURCE_SYSTEM, quoted=False),
      Identifier(this=DC_CODE, quoted=False),
      Ide

In [127]:
dependencies = {}

for cte in parse_one(sql).find_all(exp.CTE):
    dependencies[cte.alias_or_name] = []
    cte_query = cte.this.sql()

    for table in parse_one(cte_query).find_all(exp.Table):
        dependencies[cte.alias_or_name].append(table.name)

        print(dependencies)

In [191]:
from sqlglot import exp

column_list = []

# print all column references (a and b)
for column in parse_one(sql).find_all(exp.Column):
    column_list.append(column.alias_or_name)

column_list

['PO_CUT',
 'STYLE',
 'COLOR',
 'STYLE_DESCRIPTION',
 'GOODS_DESCRIPTION',
 'PO_ISSUE_DATE',
 'ORIGINAL_CRD_AT_ORIGIN',
 'REVISED_CRD_AT_ORIGIN',
 'LOCAL_CURRENCY',
 'SEASON',
 'SOURCING_OFFICE',
 'SOURCE_SYSTEM',
 'DC_CODE',
 'PO_TYPE',
 'PURCHASING_GROUP',
 'SBU',
 'SUB_SBU',
 'PRODUCT_LINE',
 'DELAY_REASON',
 'SHIPMENT_TERMS',
 'PO_LOCATION',
 'isSample',
 'isCustomOrder',
 'MISC26',
 'MISC33',
 'PURCHASING_COMPANY',
 'master_po',
 'freight_paid_by',
 'exit_cnty_port',
 'ship_to_name',
 'Brand_Requested_in_DC_date',
 'season_cleaned',
 'name',
 'ACTUAL_CRD_AT_ORIGIN',
 'pg_name',
 'managing_office',
 'src_sys',
 'created_date',
 'hts_code',
 'designation',
 'rfactory',
 'rda_code',
 'PO_Season',
 'revised_dc_date',
 'shipment_mode',
 'ack_date',
 'done_flag',
 'name',
 'PO_CUT',
 'STYLE',
 'COLOR',
 'STYLE_DESCRIPTION',
 'GOODS_DESCRIPTION',
 'PO_ISSUE_DATE',
 'ORIGINAL_CRD_AT_ORIGIN',
 'REVISED_CRD_AT_ORIGIN',
 'LOCAL_CURRENCY',
 'SEASON',
 'SOURCING_OFFICE',
 'SOURCE_SYSTEM',
 'DC

AttributeError: 'Identifier' object has no attribute 'split'

In [190]:
# find all projections in select statements (a and c)

alias_list = []

for select in parse_one(sql).find_all(exp.Select):
    for projection in select.expressions:
        alias_list.append(projection.args)

#column
alias_list[4]['this'].this

#table
alias_list[4]['table'].this

list(alias_list[5].keys())


['this', 'table', 'db', 'catalog']

In [130]:
# find all tables (x, y, z)

table_list = []
db_list = []

for table in parse_one(sql).find_all(exp.Table):
    table_list.append(table.name)
    db_list.append(table.db)
    
table_list

['otp_po_cut_level',
 'dim_fty_supr',
 'dim_fty_supr',
 'dim_pch_grp',
 'mapping_crc_grouping',
 'sp_managing_office_mapping',
 'sp_country_master',
 'mapping_ngcv12_factory',
 'ffc_factory_designation_status',
 'fact_po_hdr',
 'fact_po_dtl',
 'fact_shp_dtl',
 'fact_shp_hdr',
 'dim_size_scale',
 'fact_po_hdr',
 'fact_po_dtl',
 'fact_shp_dtl',
 'fact_shp_hdr',
 'dim_size_scale']

In [131]:
db_list

['qma_datamart',
 'qma',
 'qma',
 'qma',
 'qma_datamart',
 'qma_datamart',
 'qma_datamart',
 'qma_datamart',
 'qma',
 'qma',
 'qma',
 'qma',
 'qma',
 'qma',
 'qma',
 'qma',
 'qma',
 'qma',
 'qma']

In [161]:
from sqlglot.optimizer.scope import traverse_scope

def qualify_columns(expression, schema):
    try:      
        expression = optimizer.qualify_tables.qualify_tables(expression)
        expression = optimizer.isolate_table_selects.isolate_table_selects(expression)
        expression = optimizer.qualify_columns.qualify_columns(expression, schema)

    except (OptimizeError) as e:
        pass  
 
    return expression

def parse_statement(sql_query, dialect):

    ast = parse_one(sql_query, read=dialect)
    ast = qualify_columns(ast, schema = None)
    
    physical_columns = []
    
    for scope in traverse_scope(ast):
        for c in scope.columns:
            if isinstance(scope.sources.get(c.table), exp.Table):
                physical_columns.append((scope.sources.get(c.table).name, c.name))
    return physical_columns

parse_statement(sql, 'redshift')

[('fact_po_hdr', 'lot'),
 ('fact_po_hdr', 'style'),
 ('fact_po_dtl', 'color'),
 ('fact_po_hdr', 'desc'),
 ('fact_po_hdr', 'good_desc'),
 ('fact_po_hdr', 'issue_date'),
 ('fact_po_hdr', 'orig_date'),
 ('fact_po_hdr', 'revised_date'),
 ('fact_shp_dtl', 'actual_crd'),
 ('fact_shp_dtl', 'actual_crd'),
 ('fact_po_hdr', 'currency'),
 ('fact_po_hdr', 'currency'),
 ('fact_po_hdr', 'currency'),
 ('fact_po_hdr', 'season_cdf'),
 ('fact_po_hdr', 'source'),
 ('fact_po_hdr', 'source_sys'),
 ('fact_po_hdr', 'store_no'),
 ('fact_po_hdr', 'po_type'),
 ('fact_po_hdr', 'purchasing_grp'),
 ('fact_po_hdr', 'sbu'),
 ('fact_po_hdr', 'sub_sbu'),
 ('fact_po_hdr', 'src_sys'),
 ('fact_po_hdr', 'cls_code'),
 ('fact_po_hdr', 'src_sys'),
 ('fact_po_hdr', 'pd_line'),
 ('fact_po_hdr', 'late_exit_rsn'),
 ('fact_po_hdr', 'shipment_term'),
 ('fact_po_hdr', 'po_status'),
 ('fact_po_hdr', 'style'),
 ('fact_po_hdr', 'style'),
 ('fact_po_dtl', 'qty'),
 ('dim_size_scale', 'scale_desc'),
 ('fact_po_dtl', 'qty'),
 ('fact_po_dt

In [133]:
expression = sqlglot.parse_one(sql)
scopes = traverse_scope(expression)

len(scopes)

4

In [73]:
scopes[0].expression.sql(), list(scopes[0].sources)

("SELECT fact_po_hdr.lot AS PO_CUT, fact_po_hdr.style AS STYLE, fact_po_dtl.color AS COLOR, fact_po_hdr.desc AS STYLE_DESCRIPTION, fact_po_hdr.good_desc AS GOODS_DESCRIPTION, fact_po_hdr.issue_date AS PO_ISSUE_DATE, fact_po_hdr.orig_date AS ORIGINAL_CRD_AT_ORIGIN, fact_po_hdr.revised_date AS REVISED_CRD_AT_ORIGIN, CASE WHEN MAX(COALESCE(fact_shp_dtl.actual_crd, '9999-12-31')) = '9999-12-31' THEN NULL ELSE MAX(fact_shp_dtl.actual_crd) END AS ACTUAL_CRD_AT_ORIGIN, CASE WHEN fact_po_hdr.currency IS NULL OR fact_po_hdr.currency = '' THEN 'USD' ELSE fact_po_hdr.currency END AS LOCAL_CURRENCY, fact_po_hdr.season_cdf AS SEASON, REGEXP_REPLACE(fact_po_hdr.source, '\\s+', ' ') AS SOURCING_OFFICE, REGEXP_REPLACE(fact_po_hdr.source_sys, '\\s+', ' ') AS SOURCE_SYSTEM, REGEXP_REPLACE(fact_po_hdr.store_no, '\\s+', ' ') AS DC_CODE, fact_po_hdr.po_type AS PO_TYPE, REGEXP_REPLACE(fact_po_hdr.purchasing_grp, '\\s+', ' ') AS PURCHASING_GROUP, REGEXP_REPLACE(fact_po_hdr.sbu, '\\s+', ' ') AS SBU, REGEXP_RE

In [74]:
scopes[1].expression.sql(), list(scopes[1].sources)

("SELECT fact_po_hdr.lot AS PO_CUT, fact_po_hdr.style AS STYLE, fact_po_dtl.color AS COLOR, fact_po_hdr.desc AS STYLE_DESCRIPTION, fact_po_hdr.good_desc AS GOODS_DESCRIPTION, fact_po_hdr.issue_date AS PO_ISSUE_DATE, fact_po_hdr.orig_date AS ORIGINAL_CRD_AT_ORIGIN, fact_po_hdr.revised_date AS REVISED_CRD_AT_ORIGIN, CASE WHEN MAX(COALESCE(fact_shp_dtl.actual_crd, '9999-12-31')) IN ('9999-12-31', '1900-01-01') THEN NULL ELSE MAX(fact_shp_dtl.actual_crd) END AS ACTUAL_CRD_AT_ORIGIN, CASE WHEN fact_po_hdr.currency IS NULL OR fact_po_hdr.currency = '' THEN 'USD' ELSE fact_po_hdr.currency END AS LOCAL_CURRENCY, fact_po_hdr.season_cdf AS SEASON, REGEXP_REPLACE(fact_po_hdr.source, '\\s+', ' ') AS SOURCING_OFFICE, REGEXP_REPLACE(fact_po_hdr.source_sys, '\\s+', ' ') AS SOURCE_SYSTEM, REGEXP_REPLACE(fact_po_hdr.store_no, '\\s+', ' ') AS DC_CODE, fact_po_hdr.po_type AS PO_TYPE, REGEXP_REPLACE(fact_po_hdr.purchasing_grp, '\\s+', ' ') AS PURCHASING_GROUP, REGEXP_REPLACE(fact_po_hdr.sbu, '\\s+', ' ') 

In [134]:
# find all output column name

alias_list = []

for select in parse_one(sql).find_all(exp.Select):
    for projection in select.expressions:
        alias_list.append(projection.alias_or_name)
        
alias_list

['COUNTRY_OF_ORIGIN',
 'PO_CUT',
 'STYLE',
 'COLOR',
 'STYLE_DESCRIPTION',
 'GOODS_DESCRIPTION',
 'PO_ISSUE_DATE',
 'ORIGINAL_CRD_AT_ORIGIN',
 'REVISED_CRD_AT_ORIGIN',
 '',
 'LOCAL_CURRENCY',
 'SEASON',
 'SOURCING_OFFICE',
 'SOURCE_SYSTEM',
 'DC_CODE',
 'PO_TYPE',
 'PURCHASING_GROUP',
 'SBU',
 'SUB_SBU',
 'PRODUCT_LINE',
 'PURCHASING_COMPANY',
 'VENDOR_FFC',
 'VENDOR_GROUP_NAME',
 'FACTORY_FFC',
 'DELAY_REASON',
 'SHIPMENT_TERMS',
 'PO_LOCATION',
 'VENDOR_NAME',
 'FACTORY_NAME',
 'REPORT_ORDER_QTY_LUM',
 'ORDER_AMOUNT_LC',
 'SHIPPED_QTY_LUM',
 'SHIPPED_AMT_LC',
 'BOOKED_QTY_LUM',
 'BOOKED_AMT_LC',
 'EFFECTIVE_QTY',
 'isSample',
 'isCustomOrder',
 'MANAGING_OFFICE',
 'CURRENT_DATE',
 'src_sys',
 'MISC3',
 'MISC26',
 'MISC33',
 'PURCHASING_COMPANY',
 'exch_rate_date',
 'od_misc_flag',
 'tbr_defect_flag',
 'po_creation_date',
 'shipment_id',
 'replace',
 'factory_designation',
 'ERP_Factory_Code',
 'ERP_Vendor_Code',
 'PO_Season',
 'Revised_In_DC_Date',
 'Ship_Mode',
 'PO_Acknowledgement_

In [154]:
for node in lineage.lineage('BOOKED_QTY_LUM', sql).walk():
    
    
    #field name
    node_ = node.name
    #sql expression
    expression = node.expression
    source = node.source
    depth = expression.depth
    
    #full source
    source = node.source
    print(node.name)
    #print(expression)
    print(expression)
    print(1000*'-')

    

booked_qty_lum
SUM(CASE WHEN z.done_flag = TRUE OR z.shipped_qty_lum > 0 THEN 0 ELSE z.report_order_qty_lum END) AS booked_qty_lum
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
#find all real table
from sqlglot.optimizer.qualify import qualify

qualify(ast)

tables = [
    source

    # Traverse the Scope tree, not the AST
    for scope in root.traverse()

    # `selected_sources` contains sources that have been selected in this scope, e.g. in a FROM or JOIN clause.
    # `alias` is the name of this source in this particular scope.
    # `node` is the AST node instance
    # if the selected source is a subquery (including common table expressions),
    #     then `source` will be the Scope instance for that subquery.
    # if the selected source is a table,
    #     then `source` will be a Table instance.
    for alias, (node, source) in scope.selected_sources.items()
    if isinstance(source, exp.Table)
]

for table in tables:
    print(table)

# y  -- Success!

NameError: name 'root' is not defined

In [15]:
from sqlglot import lineage

num_of_node = 0
node_name={}
node_table={}

for node in lineage.lineage('*', sql).walk():
    
    #print(node)
    #print(1000*'--')
    
    print(f"node name is: {node.expression}")
    #print(f"node downstream is: {node.downstream}")
    #print(len(node.downstream))
    #print(f"node table is: {node.table}")
    num_of_node = num_of_node + 1

#print(f'{num_of_node}')

node name is: *
node name is: *
node name is: qma.dim_rsn_cde AS dim_rsn_cde
node name is: dim_rsn_cde.rsn_seq AS rsn_seq
node name is: qma.dim_rsn_cde AS dim_rsn_cde


In [16]:
#find all table
for table in parse_one(sql).find_all(exp.Table):
  print(f"Table => {table.name} | DB => {table.db}")

Table => DIM_RSN_CDE | DB => QMA
Table => DIM_RSN_CDE | DB => QMA


In [17]:
for column in parse_one(sql, dialect="redshift").find_all(exp.Column):
    print(f"Column => {column.name} | DB => {column.table}" )

Column => rsn_seq | DB => 
Column => rsn_id | DB => 
Column => rsn_code | DB => 
Column => rsn_desc | DB => 
Column => requestedby | DB => 
Column => created_by | DB => 
Column => created_on | DB => 
Column => modified_by | DB => 
Column => modified_on | DB => 
Column => row_vsn | DB => 
Column => rsn_code | DB => 
Column => src_sys | DB => 
Column => rsn_code | DB => 
Column => src_sys | DB => 
Column => src_sys | DB => 
Column => rsn_id | DB => 
Column => rsn_id | DB => 


In [20]:
#find dependencies of table from cte

dependencies = {}

for cte in parse_one(sql).find_all(exp.CTE):
  dependencies[cte.alias_or_name] = []

  cte_query = cte.this.sql()
  for table in parse_one(cte_query).find_all(exp.Table):
    dependencies[cte.alias_or_name].append(table.name)
print(dependencies)

{}


In [35]:
for column in parse_one(sql, dialect="redshift").find_all(exp.Column):
  print(f"Column => {column.name} | DB => {column.table}")

Column => PO_CUT | DB => z
Column => STYLE | DB => z
Column => COLOR | DB => z
Column => STYLE_DESCRIPTION | DB => z
Column => GOODS_DESCRIPTION | DB => z
Column => PO_ISSUE_DATE | DB => z
Column => ORIGINAL_CRD_AT_ORIGIN | DB => z
Column => REVISED_CRD_AT_ORIGIN | DB => z
Column => LOCAL_CURRENCY | DB => z
Column => SEASON | DB => z
Column => SOURCING_OFFICE | DB => z
Column => SOURCE_SYSTEM | DB => z
Column => DC_CODE | DB => z
Column => PO_TYPE | DB => z
Column => PURCHASING_GROUP | DB => z
Column => SBU | DB => z
Column => SUB_SBU | DB => z
Column => PRODUCT_LINE | DB => z
Column => DELAY_REASON | DB => z
Column => SHIPMENT_TERMS | DB => z
Column => PO_LOCATION | DB => z
Column => isSample | DB => z
Column => isCustomOrder | DB => z
Column => MISC26 | DB => z
Column => MISC33 | DB => z
Column => PURCHASING_COMPANY | DB => z
Column => master_po | DB => z
Column => name | DB => sp_country_master
Column => ACTUAL_CRD_AT_ORIGIN | DB => z
Column => pg_name | DB => dim_pch_grp
Column => 

In [36]:
for select in parse_one(sql).find_all(exp.Select):
    for projection in select.expressions:
        print(projection.alias_or_name)

COUNTRY_OF_ORIGIN
PO_CUT
STYLE
COLOR
STYLE_DESCRIPTION
GOODS_DESCRIPTION
PO_ISSUE_DATE
ORIGINAL_CRD_AT_ORIGIN
REVISED_CRD_AT_ORIGIN

LOCAL_CURRENCY
SEASON
SOURCING_OFFICE
SOURCE_SYSTEM
DC_CODE
PO_TYPE
PURCHASING_GROUP
SBU
SUB_SBU
PRODUCT_LINE
PURCHASING_COMPANY
VENDOR_FFC
VENDOR_GROUP_NAME
FACTORY_FFC
DELAY_REASON
SHIPMENT_TERMS
PO_LOCATION
PARTY_RESPONSIBLE
VENDOR_NAME
FACTORY_NAME
REPORT_ORDER_QTY_LUM
ORDER_AMOUNT_LC
SHIPPED_QTY_LUM
SHIPPED_AMT_LC
BOOKED_QTY_LUM
BOOKED_AMT_LC
EFFECTIVE_QTY
isSample
isCustomOrder
MANAGING_OFFICE
CURRENT_DATE
src_sys
MISC3
MISC26
MISC33
PURCHASING_COMPANY
exch_rate_date
od_misc_flag
tbr_defect_flag
po_creation_date
shipment_id
fr_release
replace
factory_designation
ERP_Factory_Code
ERP_Vendor_Code
PO_Season
Revised_In_DC_Date
Ship_Mode
PO_Acknowledgement_Date
PO_Completion_Date
PO_Complete_Status
Greenlight_date
Shipment_id_closing_date
First_Shipment_ID
Actual_Ship_Date
First_Actual_Ship_Date
First_Actual_crd
master_po
PO_CUT
STYLE
COLOR
STYLE_DESCRIP

In [59]:
ast = parse_one(sql, read='redshift')

from sqlglot.optimizer.scope import build_scope

from sqlglot.optimizer.scope import traverse_scope

root = build_scope(ast)

In [58]:
from sqlglot.optimizer.scope import find_all_in_scope

column_name = []
column_source = []

# `find_all_in_scope` is similar to `Expression.find_all`, except it doesn't traverse into subqueries
for column in find_all_in_scope(root.expression, exp.Column):
    print(f"{column} => {root.sources[column.table]}")
    #column_name.append(column)
    #column_source.append(root.sources[column.table])
    print(1000*'-')

In [21]:
tables = [
    source

    # Traverse the Scope tree, not the AST
    for scope in root.traverse()

    # `selected_sources` contains sources that have been selected in this scope, e.g. in a FROM or JOIN clause.
    # `alias` is the name of this source in this particular scope.
    # `node` is the AST node instance
    # if the selected source is a subquery (including common table expressions),
    #     then `source` will be the Scope instance for that subquery.
    # if the selected source is a table,
    #     then `source` will be a Table instance.
    for alias, (node, source) in scope.selected_sources.items()
    if isinstance(source, exp.Table)
]

for table in tables:
    print(table)

NameError: name 'root' is not defined

In [ ]:
def qualify_columns(expression, schema):
    try:      
        expression = optimizer.qualify_tables.qualify_tables(expression)
        expression = optimizer.isolate_table_selects.isolate_table_selects(expression)
        expression = optimizer.qualify_columns.qualify_columns(expression, schema)

    except (OptimizeError) as e:
        pass  
 
    return expression

In [ ]:
qualify_columns(ast, schema = None)

In [ ]:
from sqlglot.optimizer.scope import traverse_scope

def parse_statement(sql_query, dialect):

    ast = parse_one(sql_query, read=dialect)
    ast = qualify_columns(ast, schema = None)
    
    physical_columns = []
    
    for scope in traverse_scope(ast):
        print(scope)
        for c in scope.columns:
            if isinstance(scope.sources.get(c.table), exp.Table):
                physical_columns.append((scope.sources.get(c.table).name, c.name))
                
    return physical_columns

In [ ]:
parse_statement(sql, 'redshift')